In [ ]:
import torch
import cv2
import os
import pandas as pd
import numpy as np

In [ ]:
# !pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
folder_path = os.path.abspath('./tif_files')

def load_names(folder_path):
    all_files = os.listdir(folder_path)
    file_id = []
    for i, name in enumerate(all_files):
        test_file_name = name[:-4]
        file_id.append(test_file_name)
    return file_id

file_id = load_names(folder_path)

In [ ]:
challenge_img = []
for i, name in enumerate(file_id):
    file_path = os.path.join(folder_path, name+'.tif')
    img = cv2.imread(file_path, 0)
    challenge_img.append(img)
    print(i)

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

In [ ]:
all_data = pd.DataFrame()

In [ ]:
results = model(challenge_img[220])

In [ ]:
results.show()

In [ ]:
for i in range(len(challenge_img)):

    results = model(challenge_img[i])

    if len(results.pandas().xyxy[0]) > 0:
        temp_all_data = pd.DataFrame()
        temp_all_data = results.pandas().xyxy[0]
        temp_all_data['Id'] = file_id[i]
    #     temp_all_data['Id'] = file_id[i]
        all_data = all_data.append(temp_all_data, ignore_index=True)
    else:
        empty_data = pd.DataFrame([[0,0,0,0,0,100]],columns = ['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class'])
        empty_data['Id'] = file_id[i]
        all_data = all_data.append(empty_data, ignore_index=True)

    print(i)    

In [ ]:
all_data

In [ ]:
all_data.to_csv('all_class_only_img.csv', index=False)

In [ ]:
results = all_data.drop(columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence'])

In [ ]:
results

In [ ]:
results['Expected'] = np.where(results['class']==0, 1, 0)

In [ ]:
results = results.drop(columns = ['class', 'name'])

In [ ]:
results = results.drop_duplicates(subset=['Id'])

In [ ]:
results.to_csv('prediction.csv', index=False)